# 02. Data Layer Deep Dive

Iceberg 테이블의 **Data Layer**를 구성하는 Parquet 파일의 내부 구조를 직접 분석합니다.

## Data Layer 개요

Iceberg의 Data Layer는 실제 레코드가 저장된 **데이터 파일**들로 구성됩니다.

- 기본 포맷: **Apache Parquet** (ORC, Avro도 지원)
- Parquet는 **Columnar Format** (열 기반 저장)
  - 분석 쿼리에서 필요한 컬럼만 읽어 I/O를 줄임
  - 같은 컬럼의 데이터를 모아 저장하므로 **압축 효율**이 높음

### Parquet 파일 내부 구조

```
┌──────────────────────────┐
│       Row Group 1        │
│  ┌────┬────┬────┬────┐  │
│  │Col1│Col2│Col3│... │  │  ← Column Chunks
│  └────┴────┴────┴────┘  │
├──────────────────────────┤
│       Row Group 2        │
│  ┌────┬────┬────┬────┐  │
│  │Col1│Col2│Col3│... │  │
│  └────┴────┴────┴────┘  │
├──────────────────────────┤
│        Footer            │
│  (Schema, Row Group      │
│   metadata, statistics)  │
└──────────────────────────┘
```

- **Row Group**: 행의 묶음 (기본 128MB)
- **Column Chunk**: Row Group 내 단일 컬럼의 데이터
- **Footer**: 스키마, 통계 등 메타데이터

## 환경 설정

In [13]:
import sys
sys.path.append('..')

from utils.spark_setup import create_spark_session
from utils.data_generator import generate_orders, to_spark_df
from utils.file_explorer import show_tree, count_files, total_size

In [14]:
spark = create_spark_session("DataLayerDeepDive")

Spark + Iceberg 세션 준비 완료 (warehouse: file:///home/jovyan/data/warehouse)


## 테이블 생성 및 데이터 삽입

3번의 배치로 나눠 삽입하여 **파티션당 여러 개의 Parquet 파일**이 생기는 상황을 만듭니다.
같은 파티션(월)에 속하는 데이터라도 별도 append이면 별도 파일로 저장됩니다.

In [15]:
spark.sql("CREATE DATABASE IF NOT EXISTS demo.lab")
spark.sql("DROP TABLE IF EXISTS demo.lab.data_orders")

spark.sql("""
    CREATE TABLE demo.lab.data_orders (
        order_id     BIGINT,
        customer_id  BIGINT,
        product_name STRING,
        order_date   DATE,
        amount       DOUBLE,
        status       STRING
    )
    USING iceberg
    PARTITIONED BY (months(order_date))
""")

# 3번의 배치 적재 → 파티션당 최대 3개 parquet 파일
for i in range(3):
    orders = generate_orders(num_records=300, seed=i, id_offset=i*300+1)
    df = to_spark_df(spark, orders)
    df.writeTo("demo.lab.data_orders").append()
    print(f"배치 {i+1}/3 완료 (300건 삽입)")

count = spark.sql("SELECT COUNT(*) AS cnt FROM demo.lab.data_orders").collect()[0]["cnt"]
print(f"\n총 레코드: {count}건")

배치 1/3 완료 (300건 삽입)
배치 2/3 완료 (300건 삽입)
배치 3/3 완료 (300건 삽입)

총 레코드: 900건


## 디렉토리 구조 확인

In [16]:
TABLE_PATH = "/home/jovyan/data/warehouse/lab/data_orders"

show_tree(TABLE_PATH)

├── data/
│   ├── order_date_month=2024-01/
│   │   ├── 00000-11-edb76034-f263-40b1-bd24-4fb8080e2f7e-00001.parquet  (3.1 KB)
│   │   ├── 00000-17-3dd389cf-d3bd-46c0-ad35-f9f62ed6d28b-00001.parquet  (3.1 KB)
│   │   └── 00000-5-052ded78-4510-4ec0-ba2d-b983d5ef126b-00001.parquet  (3.2 KB)
│   ├── order_date_month=2024-02/
│   │   ├── 00000-11-edb76034-f263-40b1-bd24-4fb8080e2f7e-00002.parquet  (3.1 KB)
│   │   ├── 00000-17-3dd389cf-d3bd-46c0-ad35-f9f62ed6d28b-00002.parquet  (3.1 KB)
│   │   └── 00000-5-052ded78-4510-4ec0-ba2d-b983d5ef126b-00002.parquet  (3.1 KB)
│   └── order_date_month=2024-03/
│       ├── 00000-11-edb76034-f263-40b1-bd24-4fb8080e2f7e-00003.parquet  (3.0 KB)
│       ├── 00000-17-3dd389cf-d3bd-46c0-ad35-f9f62ed6d28b-00003.parquet  (3.1 KB)
│       └── 00000-5-052ded78-4510-4ec0-ba2d-b983d5ef126b-00003.parquet  (3.1 KB)
└── metadata/
    ├── 92035252-c4e6-48bd-a941-4d8c2847ef9c-m0.avro  (7.3 KB)
    ├── d3e31f53-1f8b-47cd-b78e-d96f2eedca01-m0.avro  (7.3 KB)
    ├── f8121

## Parquet 파일 찾기

`glob`을 사용하여 data/ 디렉토리 내 모든 Parquet 파일을 찾습니다.

In [17]:
import glob
import os

parquet_files = sorted(glob.glob(f"{TABLE_PATH}/data/**/*.parquet", recursive=True))

print(f"Parquet 파일 수: {len(parquet_files)}개\n")
for f in parquet_files:
    rel = os.path.relpath(f, TABLE_PATH)
    size_kb = os.path.getsize(f) / 1024
    print(f"  {rel}  ({size_kb:.1f} KB)")

Parquet 파일 수: 9개

  data/order_date_month=2024-01/00000-11-edb76034-f263-40b1-bd24-4fb8080e2f7e-00001.parquet  (3.1 KB)
  data/order_date_month=2024-01/00000-17-3dd389cf-d3bd-46c0-ad35-f9f62ed6d28b-00001.parquet  (3.1 KB)
  data/order_date_month=2024-01/00000-5-052ded78-4510-4ec0-ba2d-b983d5ef126b-00001.parquet  (3.2 KB)
  data/order_date_month=2024-02/00000-11-edb76034-f263-40b1-bd24-4fb8080e2f7e-00002.parquet  (3.1 KB)
  data/order_date_month=2024-02/00000-17-3dd389cf-d3bd-46c0-ad35-f9f62ed6d28b-00002.parquet  (3.1 KB)
  data/order_date_month=2024-02/00000-5-052ded78-4510-4ec0-ba2d-b983d5ef126b-00002.parquet  (3.1 KB)
  data/order_date_month=2024-03/00000-11-edb76034-f263-40b1-bd24-4fb8080e2f7e-00003.parquet  (3.0 KB)
  data/order_date_month=2024-03/00000-17-3dd389cf-d3bd-46c0-ad35-f9f62ed6d28b-00003.parquet  (3.1 KB)
  data/order_date_month=2024-03/00000-5-052ded78-4510-4ec0-ba2d-b983d5ef126b-00003.parquet  (3.1 KB)


## 파티션 디렉토리 구조 관찰

Iceberg의 Hidden Partitioning에 의해 `order_date_month=YYYY-MM` 형태의 디렉토리가 생성됩니다.

3번의 append로 **같은 파티션에 여러 parquet 파일**이 쌓인 것을 확인하세요.

In [18]:
data_dir = os.path.join(TABLE_PATH, "data")
partitions = sorted([d for d in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, d))])

print(f"파티션 수: {len(partitions)}개\n")
for p in partitions:
    p_path = os.path.join(data_dir, p)
    files = [f for f in os.listdir(p_path) if f.endswith(".parquet")]
    p_size = sum(os.path.getsize(os.path.join(p_path, f)) for f in files)
    print(f"  {p}: {len(files)}개 파일, {p_size / 1024:.1f} KB")

파티션 수: 3개

  order_date_month=2024-01: 3개 파일, 9.4 KB
  order_date_month=2024-02: 3개 파일, 9.3 KB
  order_date_month=2024-03: 3개 파일, 9.2 KB


## Parquet 파일 내부 분석 (pyarrow)

`pyarrow.parquet`를 사용하여 Parquet 파일의 내부 구조를 분석합니다.

In [19]:
import pyarrow.parquet as pq

# 첫 번째 Parquet 파일 분석
sample_file = parquet_files[0]
print(f"분석 파일: {os.path.relpath(sample_file, TABLE_PATH)}")
print("=" * 60)

pf = pq.ParquetFile(sample_file)
meta = pf.metadata

print(f"\n[ 기본 정보 ]")
print(f"  Parquet 포맷 버전: {meta.format_version}")
print(f"  생성 라이브러리:   {meta.created_by}")
print(f"  전체 행 수:        {meta.num_rows}")
print(f"  Row Group 수:      {meta.num_row_groups}")
print(f"  컬럼 수:           {meta.num_columns}")

분석 파일: data/order_date_month=2024-01/00000-11-edb76034-f263-40b1-bd24-4fb8080e2f7e-00001.parquet

[ 기본 정보 ]
  Parquet 포맷 버전: 1.0
  생성 라이브러리:   parquet-mr version 1.13.1 (build db4183109d5b734ec5930d870cdae161e408ddba)
  전체 행 수:        99
  Row Group 수:      1
  컬럼 수:           6


In [20]:
# 스키마 확인
print("[ Schema ]")
print(pf.schema_arrow)

[ Schema ]
order_id: int64
  -- field metadata --
  PARQUET:field_id: '1'
customer_id: int64
  -- field metadata --
  PARQUET:field_id: '2'
product_name: string
  -- field metadata --
  PARQUET:field_id: '3'
order_date: date32[day]
  -- field metadata --
  PARQUET:field_id: '4'
amount: double
  -- field metadata --
  PARQUET:field_id: '5'
status: string
  -- field metadata --
  PARQUET:field_id: '6'
-- schema metadata --
iceberg.schema: '{"type":"struct","schema-id":0,"fields":[{"id":1,"name":' + 345


In [21]:
# Row Group 분석
for rg_idx in range(meta.num_row_groups):
    rg = meta.row_group(rg_idx)
    print(f"\n[ Row Group {rg_idx} ]")
    print(f"  행 수:       {rg.num_rows}")
    print(f"  총 크기:     {rg.total_byte_size / 1024:.1f} KB")
    print(f"  컬럼 수:     {rg.num_columns}")
    
    print(f"\n  Column Chunks:")
    for col_idx in range(rg.num_columns):
        col = rg.column(col_idx)
        print(f"    [{col_idx}] {col.path_in_schema:20s} | "
              f"type={str(col.physical_type):10s} | "
              f"encodings={col.encodings} | "
              f"compressed={col.total_compressed_size} B | "
              f"uncompressed={col.total_uncompressed_size} B")


[ Row Group 0 ]
  행 수:       99
  총 크기:     3.2 KB
  컬럼 수:     6

  Column Chunks:
    [0] order_id             | type=INT64      | encodings=('BIT_PACKED', 'RLE', 'PLAIN') | compressed=174 B | uncompressed=825 B
    [1] customer_id          | type=INT64      | encodings=('BIT_PACKED', 'RLE', 'PLAIN') | compressed=268 B | uncompressed=824 B
    [2] product_name         | type=BYTE_ARRAY | encodings=('PLAIN_DICTIONARY', 'BIT_PACKED', 'RLE') | compressed=307 B | uncompressed=395 B
    [3] order_date           | type=INT32      | encodings=('PLAIN_DICTIONARY', 'BIT_PACKED', 'RLE') | compressed=203 B | uncompressed=245 B
    [4] amount               | type=DOUBLE     | encodings=('BIT_PACKED', 'RLE', 'PLAIN') | compressed=525 B | uncompressed=825 B
    [5] status               | type=BYTE_ARRAY | encodings=('PLAIN_DICTIONARY', 'BIT_PACKED', 'RLE') | compressed=172 B | uncompressed=154 B


In [22]:
# Column 통계 확인
rg = meta.row_group(0)
print("[ Column Statistics (Row Group 0) ]\n")
for col_idx in range(rg.num_columns):
    col = rg.column(col_idx)
    if col.statistics:
        stats = col.statistics
        print(f"  {col.path_in_schema}:")
        print(f"    num_values={stats.num_values}, null_count={stats.null_count}")
        if stats.has_min_max:
            print(f"    min={stats.min}, max={stats.max}")
        print()

[ Column Statistics (Row Group 0) ]

  order_id:
    num_values=99, null_count=0
    min=301, max=593

  customer_id:
    num_values=99, null_count=0
    min=128, max=997

  product_name:
    num_values=99, null_count=0
    min=AirPods, max=iPhone SE

  order_date:
    num_values=99, null_count=0
    min=2024-01-01, max=2024-01-31

  amount:
    num_values=99, null_count=0
    min=80.72, max=6981.31

  status:
    num_values=99, null_count=0
    min=cancelled, max=shipped



## 파일 크기 및 레코드 수 통계

In [23]:
print("[ 전체 Parquet 파일 통계 ]\n")

total_rows = 0
total_bytes = 0

for f in parquet_files:
    pf_tmp = pq.ParquetFile(f)
    rows = pf_tmp.metadata.num_rows
    size = os.path.getsize(f)
    total_rows += rows
    total_bytes += size
    rel = os.path.relpath(f, TABLE_PATH)
    print(f"  {rel}: {rows}행, {size / 1024:.1f} KB")

print(f"\n  합계: {total_rows}행, {len(parquet_files)}개 파일, {total_bytes / 1024:.1f} KB")
if len(parquet_files) > 0:
    print(f"  파일당 평균: {total_rows / len(parquet_files):.0f}행, {total_bytes / len(parquet_files) / 1024:.1f} KB")

[ 전체 Parquet 파일 통계 ]

  data/order_date_month=2024-01/00000-11-edb76034-f263-40b1-bd24-4fb8080e2f7e-00001.parquet: 99행, 3.1 KB
  data/order_date_month=2024-01/00000-17-3dd389cf-d3bd-46c0-ad35-f9f62ed6d28b-00001.parquet: 101행, 3.1 KB
  data/order_date_month=2024-01/00000-5-052ded78-4510-4ec0-ba2d-b983d5ef126b-00001.parquet: 109행, 3.2 KB
  data/order_date_month=2024-02/00000-11-edb76034-f263-40b1-bd24-4fb8080e2f7e-00002.parquet: 107행, 3.1 KB
  data/order_date_month=2024-02/00000-17-3dd389cf-d3bd-46c0-ad35-f9f62ed6d28b-00002.parquet: 95행, 3.1 KB
  data/order_date_month=2024-02/00000-5-052ded78-4510-4ec0-ba2d-b983d5ef126b-00002.parquet: 99행, 3.1 KB
  data/order_date_month=2024-03/00000-11-edb76034-f263-40b1-bd24-4fb8080e2f7e-00003.parquet: 94행, 3.0 KB
  data/order_date_month=2024-03/00000-17-3dd389cf-d3bd-46c0-ad35-f9f62ed6d28b-00003.parquet: 104행, 3.1 KB
  data/order_date_month=2024-03/00000-5-052ded78-4510-4ec0-ba2d-b983d5ef126b-00003.parquet: 92행, 3.1 KB

  합계: 900행, 9개 파일, 27.9 KB
  파일

## 관찰 포인트

### 1. Parquet = Columnar Format
- 스키마를 보면 각 컬럼이 독립적으로 저장됩니다
- `SELECT amount FROM ...` 같은 쿼리는 amount 컬럼만 읽으면 됩니다
- 압축 효율도 같은 타입의 데이터를 모아 저장하므로 높습니다

### 2. Column Statistics
- 각 Row Group의 각 컬럼에 `min`, `max`, `null_count` 통계가 저장됩니다
- 쿼리 엔진이 이 통계를 활용하여 불필요한 Row Group을 건너뛸 수 있습니다 (**Predicate Pushdown**)

### 3. 같은 파티션, 다른 파일
- 3번의 append로 **같은 월(파티션) 안에 3개의 parquet 파일**이 생겼습니다
- 각 파일은 서로 다른 `order_id` 범위, 다른 `min`/`max` 통계를 가집니다
- Iceberg는 이 통계를 manifest file에 기록하여 **파일 수준 프루닝**에 활용합니다

### 4. Small File Problem
- 파티션당 3개의 작은 파일이 생겼는데, append를 100번 하면 **파티션당 100개**가 됩니다
- 작은 파일이 많으면 메타데이터 오버헤드, 파일 열기/닫기 비용이 증가
- **이 문제는 `4_optimization`에서 Compaction을 통해 해결합니다**

In [24]:
spark.stop()
print("Spark 세션 종료")

Spark 세션 종료
